# `QFit` Advanced Features

Tianpu Zhao, Danyang Chen and Jens Koch

This guide will walk you through advanced features and usages for you to make full use of `qfit`. 

Note: This guide is still under developement and it will be updated as we make progress in `qfit` main dev.

## 1. Importing multiple experimental data

#### 1.1 Requirements for the data files
You may wish to fit parameters with mutiple two-tone spectroscopy data, for example in the following scenarios:
- You performed a coarse scan over a wide range of DC voltage, and performed a more refined scan near the sweet spot of the qubit to resolve the qubit frequency.
- Your device has multiple DC voltage sources to control magnetic flux and/or offset charge, and you need to determine the relation between input DC voltages and the tuning parameters, which has to be fully determined with multiple two-tone spectroscopies.

`QFit` supports loading multiple spectroscopy data files and fit against all of them. There are a few requirements for the data files being loaded together:
1. Each data file represent a single run two-tone spectroscopy experiment. It should  contains 
    - a few 2D spectroscopy arrays with the same shape (usually amplitude and phase) 
    - a few 1D arrays for axes coordinates. They should have the same length as the spectroscopy arrays' x or y dimension. When they are absent, especially when the data is just an image, `QFit` will generate the coordinate automatically.
2. All the data files used within the fitting session should contain the same coordinates' names (i.e. "voltage" for the DC voltage source axes).

With the above two requirements, `QFit` accepts various input data formats including hdf5, matlab, and images. We hope to make `QFit` as easy to use as possible. Reach out to us if you want to use `QFit` with your own data format.

#### 1.2 Importing data files
Assume you have a circuit design for fluxonium, and you have two DC flux bias lines nearby. You performed two experiments that sweeps along two different lines in the $(V_1, V_2)$ space, named `joint_qubit_twotone_ct1.h5` and `joint_qubit_twotone_ct2.h5`. The first dataset is obtained by sweeping $V_1$ and keeping $V_2$ fixed, while in the second dataset $V_1$ is fixed while $V_2$ is swept. Here we show you two ways to import these data files.

##### Method 1: provide a list of file paths for initiating the `Fit` class instance

In [ ]:
# Quantum model for a single fluxonium
import scqubits as scq

fluxonium = scq.Fluxonium(
    EJ=3.0, EC=0.9, EL=0.25, flux=0.5, cutoff=100, truncated_dim=5, id_str="Fluxonium"
)
hilbert_space = scq.HilbertSpace([fluxonium])

# spectroscopy data
data = [
    "./example_data/joint_qubit_twotone_ct1.h5",
    "./example_data/joint_qubit_twotone_ct2.h5",
]

# launch QFit.
from qfit import Fit

fit = Fit(hilbert_space, data)

This way, you directly import two figures. You may add and remove figures, and switch between tabs for viewing different figures.

<p align="center">
  <img width="400" src="resources/images/tip_file_tabs.png">
</p>  

##### Method 2: load figure after initiating `Fit` instance

You may also run `fit = Fit(hs)` directly, and a window will pop up and ask you for the file you wish to import.

#### 1.3 Configuring the data

After loading the data, you will see a panel showing <span style="color:rgb(190, 130, 250);">METADATA</span> of the data that is currently presented on the right. 

<p align="center">
  <img width="400" src="resources/images/adv_tip_metadata.png">
</p>  

`QFit` automatically detects candidate axes that are stored in the h5 file, however you need to tell `QFit` which axes are for the x-axis (the input DC voltages on flux bias lines in our case), and which axis is for the y-axis (the frequency). Since both $V_1$ and $V_2$ are swept in the two datasets, we specify `voltage_1` and `voltage_2` as x coordinates and "frequency" as the y coordinate. 
<p align="center">
  <img width="300" src="resources/images/adv_tip_select_axes.png">
</p> 

Then you will see the figures oriented correctly.
<p align="center">
  <img width="300" src="resources/images/adv_tip_correct_orientation.png">
</p> 

There are a few things to note:
- Only common coordinates in all files are shown in the selection panel
- Once selected, the data may be transposed to match the selected coordinates. 
- Multiple x coordinates are allowed, while there should be only one y coordinate (frequency)

## 2. Calibrate flux crosstalk

In our example, voltages of the input DC sources for the two flux bias lines both can contribute to the external flux of the fluxonium $\Phi_{\text{ext}}$:

$$
\Phi_{\text{ext}} = \alpha_1 V_1 + \alpha_2 V_2 + \beta .
$$

To fully calibrate the relation between the voltages $V_1$, $V_2$ and the external flux $\Phi_{\text{ext}}$, we have to determine $\alpha_1$, $\alpha_2$ and $\beta$. One such way to calibrate for these coefficients is to perform two two-tone spectroscopies that sweep along two (non-colinear) lines in the $(V_1, V_2)$ space.
Since we have three unknowns, we have to provide three sets of $(V_1, V_2, \Phi_{\text{ext}})$. Labelling them as $(V_1^{(i)}, V_2^{(i)}, \Phi_{\text{ext}}^{(i)})$, with $i = 1,2,3$, we end up with three linear equations:
$$
\Phi_{\text{ext}}^{(1)} = \alpha_1 V_1^{(1)} + \alpha_2 V_2^{(1)} + \beta \\
\Phi_{\text{ext}}^{(2)} = \alpha_1 V_1^{(2)} + \alpha_2 V_2^{(2)} + \beta \\
\Phi_{\text{ext}}^{(3)} = \alpha_1 V_1^{(3)} + \alpha_2 V_2^{(3)} + \beta .
$$

This also explains why we need two two-tone sweeps that are not co-linear: we need the above three equations not linearly-dependent, so that we can obtain a unique solution to those coefficients. This is only possible if we pick up three $(V_1, V_2)$ coordinates that are not all the same line.  

The <span style="color:rgb(190, 130, 250);">CALIBRATE</span> step helps you identify three such points. We can see in the ct1 figure there are two sweet spots that looks like $\Phi_{\text{ext}} = 0$ and $0.5$, then we extract $(V_1, V_2)$ at these sweet spots and then provide the corresponding external flux values:

<p align="center">
  <img width="600" src="resources/images/adv_tip_calibrate_1.png">
</p> 

and the third point is obtained from the ct2 figure:
<p align="center">
  <img width="600" src="resources/images/adv_tip_calibrate_2.png">
</p> 

Then you successfully calibrate for the relation between voltages and the external flux. The calibration for circuits with multiple tuning knobs can be performed in a similar way in `QFit`.

Please notice that the extracted data points for these two figures are grouped separately:
<p align="center">
  <img width="600" src="resources/images/adv_tip_extract_1.png">
</p> 
<p align="center">
  <img width="600" src="resources/images/adv_tip_extract_2.png">
</p> 

## 3. Setting panel
You can find the setting button on the bottom left corner of the main window. Once clicked, a panel will pop up with multiple tabs: visual, spectrum and fit.

#### 3.1 Visual
The visual settings may improves the image contrast and help you locate peaks easier. 
<p align="center">
    <img width="200" src="resources/images/tip_settings_visual.png">
</p>

#### 3.2 Spectrum
Options in the spectrum settings can change what transitions are plotted, and how these transitions are marked.

<p align="center">
    <img width="200" src="resources/images/tip_settings_spectrum.png">
</p>

1. TRANSITIONS: transitions that **only** excites the selected subsystem selected are labelled in color; the dashed lines are other transitions. 
2. INITIAL STATE: the initial state of the transition, can either be specified in bare state label or dressed state index.
3. EVALS COUNT: number of eigenvalues evaluated during parameter sweep.
4. POINTS ADDED: to make the transition curve look smooth, we include more sweep points for parameter sweep, in addition to those included in the extracted data.
5. PHOTONS: number of photons involved in the transition process. The transition frequency is ${|\mathrm{final\,energy} - \mathrm{initial\,energy}|}/{\mathrm{photon\,number}}$.


**Examples**:

* If you want to see the frequencies of transitions that start from the state labelled by (qubit, resonator) = (1,0) (a very good reason for this is that for fluxonium at half-flux sweet spot, the occupation probability of the state 1 may be significantly large due to small 01 transition frequency), then you may specify (1,0) as the initial state.

<p align="center">
  <img width="600" src="resources/images/initial_state_10.png">
</p>  

* If you want to see frequencies of transitions that involves two photons, then you may specify photons to be 2.

<p align="center">
  <img width="600" src="resources/images/two_photon_process.png">
</p>  

* To see sideband transitions (transitions that involve changes in bare state label of more than one subsystems, such as (1,0)->(0,1)), you may select "none selected" for the transitions option. This option highlights and labels all transitions between states with identified bare labels.

<p align="center">
  <img width="600" src="resources/images/all_transitions.png">
</p>  

#### 3.3 Fit
`QFit` uses `scipy.optimize` module to fit the data. 
Before the fitting process, you can set the optimizer configurations in the fit tab, including the optimizer to use and the tolerance of the optimizer. For more information, check out the [scipy.optimize documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html).
<p align="center">
    <img width="200" src="resources/images/tip_settings_fit.png">
</p>

The fit settings provide options for optimizers (for example, tolerance).

## Understanding the status

To be completed soon...

## General tips and strategies for parameter extraction

#### Calibration
1. You can adjust your calibration (specifically the mapped values) in pre-fit (through sliders) and fit (through optimizations).

#### Extraction 
1. **Choose your points wisely!** More points with distinctive x-value being selected = longer calculation time. You may want to select representative points for fitting.

2. For transitions, selecting bare or dressed labels means that you mark your initial and final states using bare indices (states are labelled by excitations of individual subsystems) or dressed indices (states are regarded as eigenstates of the entire system and labelled as 0, 1, 2, 3,... in ascending order of energy). 

3. If you do not know what label should be given to a group of points, you may choose the "Unknown" label, and come back to label it later once you have a better idea. 

4. Transitions may not necessarily start from the ground states; sometimes a higher-lying states are populated so the transitions from this high-lying state is activated.

#### Pre-fit
1. If your circuit system is computationally demanding (e.g. many-node circuits or a system of many coupled qubits), you may try following strategies:
    - Use smaller `truncated_dim` for each subsystem within the Hilbert space, and/or reduce number of basis states (cutoffs) of each circuit; this may have a tradeoff on accuracy, though.
    - Reduce `POINTS ADDED` in the spectrum settings. 
    - Select points in a way that distinctive x-value is less. 

2. After calibration, you may go ahead to pre-fit and adjust sliders to compute transitions and compare against two-tone spectroscopies directly, then extract and label peaks in the data.

#### Miscellaneous
1. After importing experimental data, you can always switch between different panels to perform fit.
